# Surprise Singular Value Decomposition (SVD)

TODO: INTRO HERE...

## 0 Global Settings and Imports

In [2]:
import sys
sys.path.append("../../")
import surprise
from reco_utils.dataset.url_utils import maybe_download
from reco_utils.dataset.python_splitters import python_random_split
from reco_utils.evaluation.python_evaluation import rmse, mae, rsquared, exp_var

print("System version: {}".format(sys.version))
print("Surprise version: {}".format(surprise.__version__))

System version: 3.6.0 | packaged by conda-forge | (default, Feb 10 2017, 07:08:35) 
[GCC 4.2.1 Compatible Apple LLVM 7.3.0 (clang-703.0.31)]
Surprise version: 1.0.6


In [3]:
# Parameters
# Add here if we need parameters


In [ ]:
filepath = maybe_download("http://files.grouplens.org/datasets/movielens/ml-100k/u.data", "ml-100k.data")

In [ ]:
data = pd.read_csv(filepath, sep="\t", names=["UserId", "MovieId", "Rating", "Timestamp"])
data.head()

In [ ]:
train, test = python_random_split(data)